# Testing MPO

In [1]:
%reload_ext autoreload
%autoreload 2

import rdkit
rdkit.Chem.Draw.IPythonConsole.ipython_maxProperties = -1

import dgym as dg

# load all data
path = '../../dgym-data'

deck = dg.MoleculeCollection.load(
    f'{path}/DSi-Poised_Library_annotated.sdf',
    reactant_names=['reagsmi1', 'reagsmi2', 'reagsmi3']
)

reactions = dg.ReactionCollection.from_json(
    path = f'{path}/All_Rxns_rxn_library_sorted.json',
    smarts_col = 'reaction_string',
    classes_col = 'functional_groups'
)

building_blocks = dg.datasets.disk_loader(f'{path}/Enamine_Building_Blocks_Stock_262336cmpd_20230630.sdf')
fingerprints = dg.datasets.fingerprints(f'{path}/Enamine_Building_Blocks_Stock_262336cmpd_20230630_atoms.fpb')

import torch
import pyarrow.parquet as pq
table = pq.read_table('../../dgym-data/sizes.parquet')[0]
sizes = torch.tensor(table.to_numpy())

Make oracles.

In [38]:
from dgym.envs.oracle import DockingOracle, DGLOracle, RDKitOracle
from dgym.envs.utility import ClassicUtilityFunction

with open(f'{path}/dockstring_targets/CASP3_conf.txt') as f:
    config_ = [r.strip() for r in f.readlines()][:3]
    config_ = [c.strip() for term in config_ for c in term.split('=')]
    config_ = dict(zip(config_[::2], config_[1::2]))

config = {
    'size_x': 22.5,
    'size_y': 22.5,
    'size_z': 22.5,
    'search_mode': 'detailed',
    'scoring': 'gnina',
    'seed': 5,
    **config_
}

docking_oracle = DockingOracle(
    'CASP3 affinity',
    receptor_path=f'{path}/dockstring_targets/CASP3_target.pdbqt',
    config=config
)
log_P_oracle = RDKitOracle('MolLogP')
log_S_oracle = DGLOracle('GCN_canonical_ESOL')

# create evaluators
docking_cost_function = ClassicUtilityFunction(
    docking_oracle,
    ideal=(8, 9.5),
    acceptable=(7.7, 9.5)
)

log_P_cost_function = ClassicUtilityFunction(
    log_P_oracle,
    ideal=(1, 3),
    acceptable=(-0.5, 4.5)
)

log_S_cost_function = ClassicUtilityFunction(
    log_S_oracle,
    ideal=(-2, 1),
    acceptable=(-4, 1)
)

assays = [docking_oracle, log_P_oracle, log_S_oracle]
cost_functions = [docking_cost_function, log_P_cost_function, log_S_cost_function]

Instantiate designer.

In [28]:
from dgym.envs.designer import Designer, Generator

designer = Designer(
    Generator(building_blocks, fingerprints, sizes),
    reactions,
    cache = True
)

Pick 5 random starting hits.

In [29]:
# select first molecule
import random
def select_molecule(deck):
    initial_index = random.randint(0, len(deck) - 1)
    initial_molecule = deck[initial_index]
    if len(initial_molecule.reactants) == 2 \
        and designer.match_reactions(initial_molecule):
        return initial_molecule
    else:
        return select_molecule(deck)

initial_molecules = [select_molecule(deck) for _ in range(5)]
initial_library = dg.MoleculeCollection(initial_molecules)
initial_library.update_annotations()
library = initial_library

In [ ]:
[cost(library[0]) for cost in cost_functions]

In [27]:
log_S_cost_function(1)

0.0